In [1]:
!nvidia-smi

Thu Jan 20 14:41:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:2A:00.0 Off |                  N/A |
|100%   75C    P2   338W / 350W |   5101MiB / 24268MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B4:00.0 Off |                  N/A |
| 98%   

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from tqdm import tqdm

In [3]:
tfms = {
    'trn': transforms.Compose([
        transforms.ToTensor(),
    ]),
    
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),

    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

In [4]:
trn_dataset = datasets.ImageFolder(
    root = '/home/LNE/Chest/augmented/train',
    transform = tfms['trn'],
)

val_dataset = datasets.ImageFolder(
    root = '/home/LNE/Chest/augmented/val',
    transform = tfms['val'],
)

test_dataset = datasets.ImageFolder(
    root = '/home/DB/chest_xray/test',
    transform = tfms['test'],
)

In [5]:
trn_loader = torch.utils.data.DataLoader(
    trn_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=4)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=4)

In [6]:
image_datasets = {'train' : trn_dataset,
                  'val' : val_dataset}

dataloaders = {'train' : trn_loader,
               'val' : val_loader}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
params = {
    'wandb_entity': 'lne', 
    'wandb_project': 'transfer-ResNet50',
    'save_name': 'exponentialLR',
}

In [12]:
import wandb

def init_wandb(params):
    wandb.init(entity=params['wandb_entity'], project=params['wandb_project'])
    wandb.run.name = params['save_name']
    wandb.run.save()
    config = wandb.config
    config.update(params)
    return config

In [13]:
init_wandb(params)

Error processing line 1 of /opt/conda/lib/python3.8/site-packages/vision-1.0.0-py3.8-nspkg.pth:

  Traceback (most recent call last):
    File "/opt/conda/lib/python3.8/site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 553, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored


wandb: wandb version 0.12.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


{'wandb_entity': 'lne', 'wandb_project': 'transfer-EfficientNet', 'save_name': 'exponentialLR'}

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                del inputs
                del labels

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
            if phase == "train":
                wandb.log({
                    "Train Accuracy": epoch_acc,
                    "Train Loss": epoch_loss})
            else:
                wandb.log({
                    "Validation Accuracy": epoch_acc,
                    "Validation Loss": epoch_loss})
            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

##fine tuning

In [13]:
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

model = model.to(device)
wandb.watch(model)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)

# ExponentialLR
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.5)

# # CosineAnnealingLR
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=0)

In [14]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=25)

train Loss: 0.2766 Acc: 0.8918
val Loss: 0.1194 Acc: 0.9525

train Loss: 0.0737 Acc: 0.9761
val Loss: 0.0987 Acc: 0.9650

train Loss: 0.0563 Acc: 0.9822
val Loss: 0.0882 Acc: 0.9675

train Loss: 0.0478 Acc: 0.9863
val Loss: 0.0851 Acc: 0.9700

train Loss: 0.0444 Acc: 0.9877
val Loss: 0.0836 Acc: 0.9700

train Loss: 0.0428 Acc: 0.9898
val Loss: 0.0848 Acc: 0.9700

train Loss: 0.0404 Acc: 0.9909
val Loss: 0.0831 Acc: 0.9725

train Loss: 0.0421 Acc: 0.9891
val Loss: 0.0832 Acc: 0.9700

train Loss: 0.0429 Acc: 0.9880
val Loss: 0.0832 Acc: 0.9700

train Loss: 0.0399 Acc: 0.9910
val Loss: 0.0842 Acc: 0.9700

train Loss: 0.0430 Acc: 0.9895
val Loss: 0.0830 Acc: 0.9700

train Loss: 0.0411 Acc: 0.9898
val Loss: 0.0832 Acc: 0.9700

train Loss: 0.0412 Acc: 0.9898
val Loss: 0.0836 Acc: 0.9700

train Loss: 0.0411 Acc: 0.9898
val Loss: 0.0827 Acc: 0.9700

train Loss: 0.0409 Acc: 0.9889
val Loss: 0.0825 Acc: 0.9700

train Loss: 0.0440 Acc: 0.9884
val Loss: 0.0834 Acc: 0.9700

train Loss: 0.0428 Acc: 

#test

In [15]:
torch.save(model, '/home/LNE/Chest/augmented/model/resnet50_2.pt')

In [16]:
saved_model = torch.load('/home/LNE/Chest/augmented/model/resnet50_2.pt')
saved_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [17]:
running_loss = 0.0
running_corrects = 0

for inputs, labels in test_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = saved_model(inputs)
    _, preds = torch.max(outputs, 1)
    loss = criterion(outputs, labels)
    

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    del inputs
    del labels

loss = running_loss / len(test_dataset)
acc = running_corrects.double() / len(test_dataset)

wandb.log({
        "Test Accuracy": acc,
        "Test Loss": loss})
    
print('{} Loss: {:.4f} Acc: {:.4f}'.format("test", loss, acc))            

test Loss: 0.2084 Acc: 0.9216
